In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

# Loading data

In [11]:
keywords = pd.read_csv('keywords.csv')
sites = pd.read_csv('sites.csv')
rankings = pd.read_csv('rankings.csv')
universities = pd.read_csv('../../universities/university_list_countries.csv')
keyword_data = pd.read_csv('../Keyword Planner type 1 2017-01-09 at 01-28-48.csv', sep='\t')

rankings = rankings.merge(keywords, on='id_keyword')
rankings = rankings.merge(sites, on='id_site')
rankings = rankings.merge(universities, left_on='domain', right_on='Website')
rankings = rankings[['rank','keyword', 'domain', 'University', 'Country']]
rankings = rankings.drop_duplicates()  # Note the data is dirty and contains about 10000 duplicates
rankings.head()

,rank,keyword,domain,University,Country
0,43,bachelor's degree in Economics,www.au.dk,Aarhus University,Denmark
1,42,bachelor's degree in Economics,www.au.dk,Aarhus University,Denmark
2,91,bachelor's degree in English Language,www.au.dk,Aarhus University,Denmark
3,29,bachelor's degree in Archaeology,www.au.dk,Aarhus University,Denmark
4,7,bachelor's degree in Business & Commercial Law,www.au.dk,Aarhus University,Denmark


Each entry represents an appeareance of the given university domain amoung the first 100 Google search results obtained with the corresponding query (keyword). The rank is the position the university appeared on the results page.

# Initial data exploration

In [46]:
rankings.domain.describe()

count           24491
unique           1448
top       www.asu.edu
freq              223
Name: domain, dtype: object

In [47]:
sites.count()

id_site    9260
domain     9258
dtype: int64

We see that 1448 / 9260 of the universities considered appear at least once in the google search results. Seen under a different light, this means that 7812 universities are hardly visible on the web...

With unweighted results we see the ase.edu (Arizona state university) appears the most (for 223 keywords out of a possible 339)

We can see which universities are the most present in Google search results simply by seeing on how many queries they appear (see following table).

Next steps will be:
- weight the result with the rank attribute
- weight the keywords with search volume and cpc
- divide rankings into categories (degree type, subject, country etc..)

In [49]:
rankings.University.value_counts().head()

Arizona State University         223
Boston University                209
New York University              194
Stanford University              180
Pennsylvania State University    174
Name: University, dtype: int64

In [51]:
rankings.University.value_counts().to_csv('unweighted_ranking.csv')

# Keyword data

In [12]:
keyword_data.head()

,Keyword,Avg. Monthly Searches (exact match only),Competition,Suggested bid
0,phd in management,1K – 10K,0.29,3.62
1,phd in psychology,1K – 10K,0.55,24.62
2,phd in economics,1K – 10K,0.15,2.70
3,phd in germany,1K – 10K,0.26,0.98
4,phd in law,1K – 10K,0.18,3.28


# Aggregating

We invert the rank because a higher rank is better (i.e. ranking first is better than ranking 99th). We need to make sure this is reflected in the weighting.
We then weight our rankings depending on where the university came in the the search results for each query.

In [26]:
rankings['inv_rank'] = rankings['rank'].apply(lambda x: 1./x)
rankings.head()

,rank,keyword,domain,University,Country,inv_rank
0,43,bachelor's degree in Economics,www.au.dk,Aarhus University,Denmark,0.023256
1,42,bachelor's degree in Economics,www.au.dk,Aarhus University,Denmark,0.023810
2,91,bachelor's degree in English Language,www.au.dk,Aarhus University,Denmark,0.010989
3,29,bachelor's degree in Archaeology,www.au.dk,Aarhus University,Denmark,0.034483
4,7,bachelor's degree in Business & Commercial Law,www.au.dk,Aarhus University,Denmark,0.142857


In [41]:
grouped_rankings = rankings.groupby(['University','Country'])

In [60]:
grouped_weighted_rankings = grouped_rankings.sum().sort_values(['inv_rank'], ascending=False)
grouped_weighted_rankings.head()

,,rank,inv_rank
University,Country,,
Boston University,Massachusetts,7547,25.329307
New York University,New York,7445,15.952792
Stanford University,California,6479,15.136372
Southern New Hampshire University,New Hampshire,2788,14.145092
Columbia University,New York,4861,14.031654


Above is a weighted ranking of the universities

In [67]:
grouped_weighted_rankings.to_csv('weighted_rankings.csv', encoding='utf-8')

# Country by country

In [69]:
weighted_rankings = pd.read_csv('weighted_rankings.csv', encoding='utf-8')
weighted_rankings.head()

,University,Country,rank,inv_rank
0,Boston University,Massachusetts,7547,25.329307
1,New York University,New York,7445,15.952792
2,Stanford University,California,6479,15.136372
3,Southern New Hampshire University,New Hampshire,2788,14.145092
4,Columbia University,New York,4861,14.031654


In [97]:
states = ['New York', 'California', 'Pennsylvania', 'Texas', 'Massachusetts','Illinois','Ohio','Florida','Virginia','Michigan','North Carolina',
         'Tennessee','Indiana','Wisconsin','Georgia','Missouri','Minnesota','Colorado','New Jersey','Alabama','Louisiana','Washington',
         'Maryland','Oregon','Kentucky','Connecticut','Kansas','Nebraska','Oklahoma','South Carolina','Maine','Vermont','Mississippi',
         'Rhode Island','Arizona','Arkansas','District of Columbia','Iowa','Utah','New Hampshire','South Dakota','Idaho','New Mexico',
         'Delaware','Alaska','Hawaii', 'North Dakota', 'Wyoming','Montana','Nevada','West Virginia']
print(len(states))
weighted_rankings['Country'].replace(to_replace=states,value='United States of America').value_counts().head(20)

51


United States of America    898
United Kingdom               93
Canada                       55
Germany                      40
Australia                    33
Italy                        30
Spain                        20
France                       17
China                        14
Netherlands                  14
Sweden                       13
South Africa                 12
India                        12
Malaysia                     12
Pakistan                     11
Ireland                      11
Finland                       9
Japan                         9
Kenya                         8
Austria                       8
Name: Country, dtype: int64

In [96]:
weighted_rankings.replace(to_replace=states,value='United States of America').groupby('Country').size().shape

(74,)

First observation: 74 countries make it onto our rankings.
Second observation: Heavy dominance by US then english speaking countries. 
Up to here, all the queries have been made in English, so it's not surprising that the results are dominated by English speaking countries. Nevertheless many non native English universities often have websites in English, allowing them to rank. We might make adjustment in the future to balance out the playing field. However, we should note that other university rankings are heavily dominated by English speaking universities too.

# By degree type

# Technical ranking

# keyword weighting